# Assignment 2: The Winter is here
##### This works best with epic battle music. Spoilers present.
<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Tywin Lannister was right when he said: "The great war is between death and life, ice and fire. If we loose, the night will never end"<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;It has been six months since the white walkers' army marched into the north, led by the night king himself on a dead dragon. It has been a battle like never before: never before have men faced such an enemy in battle, never before have men fought so bravely against a united threat, and never before have they been so gravely defeated.<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; While Cersei is in King's landing, brave men have died fighting the great war. Among others, Tyrion is dead, Arya is dead and Jon Snow is dead, again. In a desperate battle, Daenerys leads all her forces in a final stand-off with the dead just south of Winterfell. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Her army defeated, she is now on the run on her dragon in an air battle, being chased by two of her own dragons, the Night king and a dead Jon Snow. Suddenly, the Night king's spear hits Danny's dragon, who, raining blood and fire, falls into ice, taking the lost queen, with him. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Daenerys opens her eyes in a strange place, a place which does not follow the rules of space and time, where the dead souls killed by the dead men are trapped, forever. But who woke her up? There stands near her, Tyrion, with Jorah, Davos, Jon Snow, and everybody else. They all indulge in a heartfelt reunion when someone yells- "But how do we get out?<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Varys sees a talking crystal close by, who asks them of completing a task, which on completion would allow them to go back to the land of the living, with the ultimate tool to defeat the white-walkers and kills the night king, the Dragon-axe. They have summoned you for help, as the task is out of their expertise, to apply a modified CNN to solve the object detection problem on the PASCAL VOC dataset. Varys, the master of whisperers, has used his talents to import the following for you:

In [1]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()
# You can ask Varys to get you more if you desire
import xml.etree.ElementTree as ET
import PIL.Image
import glob
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

resnet_input = 224 #size of resnet18 input images, Took it from resnet paper implementation
use_gpu = False

In [20]:
# Cersei chose violence, you choose your hyper-parameters wisely using validation data!
batch_size = 1
num_epochs = 5
learning_rate =  0.001
hyp_momentum = 0.9

## Build the data
The hound who was in charge for getting the data, brought you the following links:
<br/>Training and validation:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
<br/>Testing data:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
<br/>He also told you that the dataset(datascrolls :P) consists of images from of 20 classes, with detection annotations included. The JPEGImages folder houses the images, and the Annotations folder has the object-wise labels for the objects in one xml file per image. You have to extract the object information, ie. the [xmin, ymin] (the top left x,y co-ordinates) and the [xmax, ymax] (the bottom right x,y co-ordinates) of only the objects belonging to the given 20 classes(aeroplane, bicycle, boat, bottle, bus, car, cat, chair, cow, dining table, dog, horse, motorbike, person, potted plant, sheep, train, TV). For parsing the xml file, you can ask Varys to import xml.etree.ElementTree for you. <br/>
<br/> You can then ask Bronn and Jamie to organize the data as follows:
<br/> For every image in the dataset, extract/crop the object patch from the image one by one using their respective co-ordinates:[xmin, ymin, xmax, ymax], resize the image to resnet_input, and store it with its class label information. Do the same for training/validation and test datasets. <br/>
##### Important
You also have to collect data for an extra background class which stands for the class of an object which is not a part of any of the 20 classes. For this, you can crop and resize any random patches from an image. A good idea is to extract patches that have low "intersection over union" with any object present in the image frame from the 20 Pascal VOC classes. The number of background images should be roughly around those of other class objects' images. Hence the total classes turn out to be 21. This is important for applying the sliding window method later.

In [21]:
classes = ('__background__',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')


In [22]:
def jamie_bronn_build_dataset(box_position, imdir, filename, image):
    # Begin
    image = PIL.Image.open(imdir+filename)
    image = image.crop(box_position)
#     image = image.resize(resnet_input)
    return image

In [23]:
class hound_dataset(torch.utils.data.Dataset): # Extend PyTorch's Dataset class
    def __init__(self, root_dir, train, transform=None):
        # Begin
        self.image = []
        self.class_name = []
        if(train):
            dir = root_dir + 'train/VOCdevkit/VOC2007/'
        else :
            dir = root_dir + 'test/VOCdevkit/VOC2007/'
        
        anotdir = dir + 'Annotations/'
        imdir   = dir + 'JPEGImages/'
        
        for filepath in glob.glob(anotdir+'*.xml'):
            
            tree = ET.parse(filepath)
            root = tree.getroot()
            filename = root.find('filename').text
            
            for objects in root.findall('object'):
                class_name = objects.find('name').text
                box_position = [int(objects.find('bndbox').find('xmin').text), \
                                int(objects.find('bndbox').find('ymin').text), \
                                int(objects.find('bndbox').find('xmax').text), \
                                int(objects.find('bndbox').find('ymax').text)]
                image = jamie_bronn_build_dataset(box_position, imdir, filename, self.image)
                print("In image "+filename+" found "+class_name)
                self.image.append(image.convert('RGB'))
                # every class name corresponds to some number in the classes array that we add to the class_name array
                class_name = classes.index(class_name)
                self.class_name.append(class_name)
        self.transform = transform

    def __len__(self):
        # Begin
        # return the size of the dataset (total number of images) as an integer
        # this should be rather easy if you created a mapping in __init__
        return len(self.image)
        
    def __getitem__(self, idx):
        # Begin
        if self.transform is None:
            return (self.image[idx], self.class_name[idx])
        else:
            img_transformed = self.transform(self.image[idx])
            return (img_transformed, self.class_name[idx])
    

In [24]:
# Sanity check

## Train the netwok
<br/>You can ask Arya to train the network on the created dataset. This will yield a classification network on the 21 classes of the VOC dataset. 

In [26]:
composed_transform = transforms.Compose([transforms.Scale((resnet_input,resnet_input)),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor()])
train_dataset = hound_dataset(root_dir='/home/eeshangd/vision_env/LELImproved/dataset/', train=True, transform=composed_transform) # Supply proper root_dir
test_dataset = hound_dataset(root_dir='/home/eeshangd/vision_env/LELImproved/dataset/', train=False, transform=composed_transform) # Supply proper root_dir

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In image 005906.jpg found bus
In image 005906.jpg found person
In image 005906.jpg found person
In image 006141.jpg found bottle
In image 007219.jpg found tvmonitor
In image 007219.jpg found chair
In image 003260.jpg found tvmonitor
In image 003260.jpg found person
In image 003260.jpg found person
In image 003260.jpg found person
In image 005629.jpg found chair
In image 005629.jpg found chair
In image 005629.jpg found chair
In image 005629.jpg found chair
In image 005629.jpg found diningtable
In image 004438.jpg found dog
In image 004438.jpg found person
In image 004438.jpg found person
In image 004438.jpg found tvmonitor
In image 001125.jpg found car
In image 001125.jpg found car
In image 001125.jpg found car
In image 001125.jpg found person
In image 001125.jpg found person
In image 006802.jpg found train
In image 006802.jpg found person
In image 006802.jpg found person
In image 003254.jpg found bird
In image 007187.jpg found sofa
In image 007187.jpg found person
In image 007187.jpg f

In image 007901.jpg found person
In image 007901.jpg found person
In image 007901.jpg found person
In image 007901.jpg found person
In image 007901.jpg found person
In image 005124.jpg found cow
In image 005124.jpg found person
In image 001014.jpg found dog
In image 001014.jpg found dog
In image 001014.jpg found person
In image 001014.jpg found chair
In image 002311.jpg found car
In image 002311.jpg found car
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 002311.jpg found person
In image 003905.jpg found bird
In image 002621.jpg found bi

In image 005471.jpg found person
In image 005471.jpg found person
In image 005471.jpg found bottle
In image 007571.jpg found person
In image 007571.jpg found person
In image 007571.jpg found bicycle
In image 007571.jpg found bicycle
In image 001995.jpg found chair
In image 008784.jpg found car
In image 008784.jpg found car
In image 008784.jpg found car
In image 008784.jpg found car
In image 008784.jpg found car
In image 008784.jpg found car
In image 008784.jpg found car
In image 008784.jpg found car
In image 008784.jpg found car
In image 008784.jpg found car
In image 008784.jpg found boat
In image 008784.jpg found boat
In image 008784.jpg found boat
In image 008784.jpg found boat
In image 008784.jpg found boat
In image 008784.jpg found boat
In image 008784.jpg found boat
In image 008784.jpg found car
In image 009255.jpg found horse
In image 005875.jpg found motorbike
In image 005542.jpg found person
In image 005542.jpg found person
In image 005542.jpg found person
In image 005542.jpg f

In image 007812.jpg found chair
In image 007812.jpg found chair
In image 004708.jpg found cat
In image 004708.jpg found cat
In image 004708.jpg found sofa
In image 008691.jpg found train
In image 009545.jpg found cow
In image 001901.jpg found chair
In image 001901.jpg found chair
In image 001901.jpg found chair
In image 001901.jpg found diningtable
In image 001901.jpg found sofa
In image 003614.jpg found bird
In image 001541.jpg found cat
In image 009036.jpg found bicycle
In image 009036.jpg found bicycle
In image 009036.jpg found bicycle
In image 009036.jpg found person
In image 009036.jpg found person
In image 009036.jpg found person
In image 005438.jpg found chair
In image 005438.jpg found sofa
In image 009664.jpg found bottle
In image 003038.jpg found train
In image 004789.jpg found horse
In image 004789.jpg found person
In image 007200.jpg found motorbike
In image 007200.jpg found person
In image 003343.jpg found dog
In image 004783.jpg found bird
In image 000686.jpg found motorbi

In image 000822.jpg found motorbike
In image 000822.jpg found motorbike
In image 001501.jpg found bicycle
In image 001501.jpg found person
In image 008608.jpg found diningtable
In image 006515.jpg found bicycle
In image 008833.jpg found person
In image 008833.jpg found person
In image 007308.jpg found horse
In image 007308.jpg found person
In image 007308.jpg found horse
In image 007308.jpg found person
In image 005006.jpg found bird
In image 005212.jpg found motorbike
In image 005212.jpg found chair
In image 005212.jpg found sofa
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004585.jpg found cow
In image 004432.jpg found cat
In image 006542.jpg found person
In image 0

...

In image 003689.jpg found train
In image 007163.jpg found person
In image 007163.jpg found person
In image 007163.jpg found horse
In image 007163.jpg found horse
In image 004502.jpg found cat
In image 005824.jpg found person
In image 006278.jpg found train
In image 008023.jpg found bus
In image 008023.jpg found person
In image 008023.jpg found person
In image 008023.jpg found person
In image 008023.jpg found bicycle
In image 004514.jpg found tvmonitor
In image 005196.jpg found pottedplant
In image 005196.jpg found cat
In image 005196.jpg found person
In image 005196.jpg found person
In image 005196.jpg found sofa
In image 005196.jpg found chair
In image 005196.jpg found chair
In image 000597.jpg found person
In image 000597.jpg found person
In image 000597.jpg found person
In image 000597.jpg found chair
In image 000597.jpg found chair
In image 000597.jpg found chair
In image 000597.jpg found chair
In image 000597.jpg found chair
In image 000597.jpg found tvmonitor
In image 000597.jpg 

In image 000196.jpg found chair
In image 003544.jpg found bottle
In image 003544.jpg found bottle
In image 003544.jpg found sofa
In image 007223.jpg found cat
In image 002442.jpg found diningtable
In image 002442.jpg found chair
In image 002442.jpg found chair
In image 002442.jpg found chair
In image 002442.jpg found chair
In image 002442.jpg found chair
In image 002442.jpg found chair
In image 002442.jpg found chair
In image 002442.jpg found chair
In image 002442.jpg found bottle
In image 002442.jpg found bottle
In image 001658.jpg found car
In image 001658.jpg found motorbike
In image 001658.jpg found car
In image 001658.jpg found car
In image 001658.jpg found car
In image 009794.jpg found cat
In image 006115.jpg found sofa
In image 006115.jpg found sofa
In image 006115.jpg found person
In image 006115.jpg found pottedplant
In image 006340.jpg found car
In image 006340.jpg found person
In image 006340.jpg found person
In image 006340.jpg found person
In image 007356.jpg found bicycle

In image 003049.jpg found person
In image 003049.jpg found person
In image 004799.jpg found bird
In image 006898.jpg found person
In image 006898.jpg found person
In image 000421.jpg found tvmonitor
In image 000421.jpg found person
In image 006985.jpg found cat
In image 004638.jpg found train
In image 008094.jpg found bottle
In image 008094.jpg found person
In image 008094.jpg found person
In image 009077.jpg found person
In image 009077.jpg found person
In image 009077.jpg found person
In image 009077.jpg found person
In image 009077.jpg found person
In image 009077.jpg found person
In image 009077.jpg found person
In image 009077.jpg found car
In image 009077.jpg found car
In image 009077.jpg found car
In image 009077.jpg found car
In image 000444.jpg found boat
In image 000444.jpg found person
In image 008259.jpg found car
In image 008259.jpg found car
In image 008259.jpg found car
In image 008259.jpg found person
In image 008259.jpg found dog
In image 008259.jpg found horse
In imag

In image 004290.jpg found car
In image 002126.jpg found cow
In image 002126.jpg found cow
In image 001413.jpg found bird
In image 001413.jpg found bird
In image 001413.jpg found bird
In image 001413.jpg found bird
In image 001413.jpg found bird
In image 001413.jpg found bird
In image 001413.jpg found bird
In image 001413.jpg found bird
In image 001413.jpg found bird
In image 007732.jpg found chair
In image 007732.jpg found chair
In image 007732.jpg found tvmonitor
In image 003016.jpg found bus
In image 003016.jpg found person
In image 000887.jpg found aeroplane
In image 001327.jpg found person
In image 001460.jpg found person
In image 001460.jpg found bicycle
In image 009444.jpg found chair
In image 009444.jpg found chair
In image 009444.jpg found chair
In image 009444.jpg found chair
In image 009444.jpg found chair
In image 009444.jpg found diningtable
In image 009444.jpg found diningtable
In image 007374.jpg found car
In image 007374.jpg found car
In image 003780.jpg found tvmonitor


In image 004430.jpg found bicycle
In image 000952.jpg found person
In image 000952.jpg found person
In image 000952.jpg found person
In image 000952.jpg found person
In image 005812.jpg found dog
In image 005812.jpg found dog
In image 005411.jpg found tvmonitor
In image 005411.jpg found person
In image 003201.jpg found cow
In image 003201.jpg found cow
In image 003201.jpg found cow
In image 002830.jpg found motorbike
In image 002830.jpg found motorbike
In image 002830.jpg found motorbike
In image 002830.jpg found motorbike
In image 002830.jpg found person
In image 009590.jpg found car
In image 009590.jpg found person
In image 009590.jpg found person
In image 009590.jpg found person
In image 009590.jpg found car
In image 007485.jpg found dog
In image 007485.jpg found sofa
In image 009630.jpg found cat
In image 009630.jpg found pottedplant
In image 009630.jpg found pottedplant
In image 009630.jpg found pottedplant
In image 009630.jpg found bottle
In image 009630.jpg found bottle
In image

### Fine-tuning
Litlefinger has brought you a pre-trained network. Fine-tune the network in the following section:

In [27]:
resnet18 = models.resnet18(pretrained=True)

resnet18.fc = nn.Linear(resnet18.fc.in_features, 21)

# Add code for using CUDA here
if(torch.cuda.is_available()):
    use_gpu = True
    resnet18.cuda()

RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCStorage.cu:66

In [ ]:
criterion = nn.CrossEntropyLoss()
# Update if any errors occur
optimizer = optim.SGD(resnet18.parameters(), learning_rate, hyp_momentum)

In [ ]:
def arya_train():
    # Begin
    loss_arrr = []
    for epoch in range(num_epochs):
        for i, (image, class_name) in enumerate(train_loader):  
            # Convert torch tensor to Variable
            images = Variable(image)
            names = Variable(class_name)
            if(use_gpu):
                images=images.cuda()
                names=names.cuda()
            # Forward + Backward + Optimize
            optimizer.zero_grad()  # zero the gradient buffer
            outputs = resnet18(images)
            loss = criterion(outputs, names)
            loss.backward()
            loss_arrr.append(loss.data[0])
            optimizer_resnet18.step()
            if (i+1) % batch_size == 0:
                print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                       %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

In [ ]:
%time arya_train()

# Testing and Accuracy Calculation
Jorah then asks a question, how is this a detection task?<br/>
As everybody wonders, Theon Greyjoy suggests a slding window method to test the above trained trained network on the detection task:<br/>
"We take some windows of varying size and aspect ratios", he mumbled, "and slide it through the test image (considering some stride of pixels) from left to right, and top to bottom, detect the class scores for each of the window, and keep only those which are above a certain threshold value!". "He is right", says Samwell, "I read a similar approach in the paper -Faster RCNN by Ross Girshick in the library, where he uses three diferent scales/sizes and three different aspect ratios, making a total of nine windows per pixel to slide". You need to write the code and use it in testing code to find the predicted boxes and their classes.

In [ ]:
def theon_sliding_window():
    # Begin

"Wait", says <b>Jon Snow</b>, "The predicted boxes may be too many and we can't deal with all of them. So, I myself will go and apply non_maximum_supression to reduce the number of boxes". You are free to choose the threshold value for non maximum supression, but choose wisely [0,1].

In [ ]:
def aegon_targaryen_non_maximum_supression(boxes,threshold = 0.3):
    # 

Daenerys, the queen, then orders her army to test out the trained model on the test dataset.

In [ ]:
def daenerys_test(resnet18):
    # Write loops for testing the model on the test set
    # Also print out the accuracy of the model

In [ ]:
%time daenerys_test(resnet18)

# Final Showdown
After covering all the steps and passing the accuracy value to the talking crystal, they all pass through to the land of the living, with a wounded Jon Snow armed with the Dragon-axe. After a fierce battle, Jon Snow manages to go face to face with the Night king. Surrounded by battling men and falling bodies, they engage in a ferocious battle, a battle of spear and axe. After a raging fight, Jon manages to sink the axe into the Night king's heart, but not before he gets wounded by the spear. As dead men fall to bones, Daenerys and others rush to his aid, but it is too late. Everyone is in tears as they look towards the man of honour, Jon Snow, lying in Daenerys's arms when he says his last words: "The night has ended. Winter is finally over!"

In [7]:
print ("The night has ended. Winter is finally over!")

The night has ended. Winter is finally over!
